In [1]:
# Import Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
# Load the Size History by County csv to a DataFrame
size_history_data_path = "../cleaned_data/Electric_Vehicle_Population_Size_History_By_County_Cleaned.csv"
size_history_full_data = pd.read_csv(size_history_data_path)

In [3]:
size_history_full_data.head()

,Date,County,State,Vehicle Primary Use,Battery Electric Vehicles (BEVs),Plug-In Hybrid Electric Vehicles (PHEVs),Electric Vehicle (EV) Total,Non-Electric Vehicle Total,Total Vehicles,Percent Electric Vehicles
0,January 31 2017,Juneau,AK,Passenger,0,1,1,49,50,2.00
1,January 31 2017,Dale,AL,Passenger,0,1,1,71,72,1.39
2,January 31 2017,Russell,AL,Passenger,1,0,1,53,54,1.85
3,January 31 2017,Maricopa,AZ,Passenger,2,0,2,1774,1776,0.11
4,January 31 2017,Pima,AZ,Passenger,0,1,1,439,440,0.23


In [6]:
# Refine to most recent date in history (October 31 2021)
size_history_data = size_history_full_data.loc[size_history_full_data["Date"]=="October 31 2021"]
size_history_data

,Date,County,State,Vehicle Primary Use,Battery Electric Vehicles (BEVs),Plug-In Hybrid Electric Vehicles (PHEVs),Electric Vehicle (EV) Total,Non-Electric Vehicle Total,Total Vehicles,Percent Electric Vehicles
11420,October 31 2021,Fairbanks North Star,AK,Passenger,0,1,1,62,63,1.59
11421,October 31 2021,Montgomery,AL,Passenger,0,1,1,30,31,3.23
11422,October 31 2021,Maricopa,AZ,Passenger,1,0,1,597,598,0.17
11423,October 31 2021,Alameda,CA,Passenger,1,0,1,157,158,0.63
11424,October 31 2021,Fresno,CA,Passenger,1,0,1,128,129,0.78
...,...,...,...,...,...,...,...,...,...,...
11609,October 31 2021,Whitman,WA,Passenger,69,57,126,21414,21540,0.58
11610,October 31 2021,Whitman,WA,Truck,2,0,2,8338,8340,0.02
11611,October 31 2021,Yakima,WA,Passenger,282,166,448,163726,164174,0.27
11612,October 31 2021,Yakima,WA,Truck,5,0,5,59634,59639,0.01


In [8]:
# Copy and group dataframe into Counties, then summarise columns of interest
county_data_EV_count = size_history_data.groupby("County")["Electric Vehicle (EV) Total"].sum()
county_data_total_count = size_history_data.groupby("County")["Total Vehicles"].sum()
county_data_percent = (county_data_EV_count / county_data_total_count)*100
county_data_percent = county_data_percent.map("{:.2f}".format)

# Assemble columns into dataframe
county_data = pd.DataFrame(
    {"Total Electric Vehicle (EV)": county_data_EV_count,
     "Total Vehicles": county_data_total_count,
     "Percent Electric Vehicles": county_data_percent
    }
)

# Sort data by descending order
county_data_sorted_percent = county_data.sort_values("Percent Electric Vehicles", ascending=False)
county_data_sorted_count = county_data.sort_values("Total Electric Vehicle (EV)", ascending=False)

# Display data
county_data_sorted_count

,Total Electric Vehicle (EV),Total Vehicles,Percent Electric Vehicles
County,,,
King,43352,1560184,2.78
Snohomish,8933,657596,1.36
Pierce,6307,693060,0.91
Clark,4817,399728,1.21
Kitsap,3005,233287,1.29
...,...,...,...
New Castle,1,12,8.33
Muscogee,1,118,0.85
Dupage,1,27,3.70


In [10]:
# Copy and group dataframe into States, then summarise columns of interest
state_data_EV_count = size_history_data.groupby("State")["Electric Vehicle (EV) Total"].sum()
state_data_total_count = size_history_data.groupby("State")["Total Vehicles"].sum()
state_data_percent = (state_data_EV_count / state_data_total_count)*100
state_data_percent = state_data_percent.map("{:.2f}".format)

# Assemble columns into dataframe
state_data = pd.DataFrame(
    {"Total Electric Vehicle (EV)": state_data_EV_count,
     "Total Vehicles": state_data_total_count,
     "Percent Electric Vehicles": state_data_percent
    }
)

# Sort data by descending order
state_data_sorted_percent = state_data.sort_values("Percent Electric Vehicles", ascending=False)
state_data_sorted_count = state_data.sort_values("Total Electric Vehicle (EV)", ascending=False)

# Display data
state_data_sorted_count

,Total Electric Vehicle (EV),Total Vehicles,Percent Electric Vehicles
State,,,
WA,82851,6099984,1.36
CA,47,5012,0.94
VA,33,2270,1.45
MD,25,894,2.80
TX,15,1304,1.15
NC,8,722,1.11
OR,8,465,1.72
GA,7,491,1.43
FL,6,486,1.23


**State Findings**

Washington has the highest count of electric vehicles with 82851 vehicles, making up 1.36 percent of their total vehicles.

**County Findings**

King, Washington has the highest count of electric vehicles with 43352 vehicles, making up 2.78 percent of their total vehicles.

It was noted that there is far more data available for Washington and its counties, when compared to the other states and counties - e.g. Wisconsin only has 4 total vehicles (both electrical and non-electrical). Because of this, I believe that the dataframe does not include all vehicle data for other states, and thus we should consider removing other state data from our analysis as it may lead to incorrect insights.